In [1]:
# SAMMY DO NOT RUN, this is just for my venv
import sys
sys.path.append('/Users/tunadorable/local-repos/next-concept-predictor/venv/lib/python3.11/site-packages')

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Table of contents

make sure to read comments & whatnot to keep track of what's happening. the accompanying pdf is very important for a conceptual background

1. recap on the basic stuff (bulk of the transformer) which remains unchanged
2. recap on what a regular GPT output layer looks like
3. recap on how a regular GPT's training works
4. recap on how a regular GPT's inference works
5. my model's output layer
6. my model's weird conditional logic
7. my model's training
8. my model's inference
- 8a. first we'll get the user their output tokens (in batch)
- 8b. then we'll create what the model is going to want for its next round of inference
9. Turning everything into useful functions with printouts
- 9a. gpt initialization
- 9b. gpt guts
- 9c. output/training/inference components of regular GPTs which we will not be using. I only wrote them out here for demonstration purposes.
- 9d. my model's output layer
- 9e. my model's weird conditional logic
- 9f. my model's training
- 9g. my model's inference process
10. making a GPT training loop
11. making an GPT inference loop
12. training loop for my model
13. inference loop for my model

# 1. the basic stuff / bulk of the transformer

this section is left unchanged

In [3]:
# setting hyperparameters
b=2
n=7
d=4
v=12
seed = 69

In [4]:
E_dict = {" I":0,
          " think":1,
         " there":2,
         "fore":3,
         " am":4,
         "Every":5,
         " cloud":6,
         " has":7,
         " a":8,
         " silver":9,
         " lining":10,
         "<endoftext>":11}
E_dict

{' I': 0,
 ' think': 1,
 ' there': 2,
 'fore': 3,
 ' am': 4,
 'Every': 5,
 ' cloud': 6,
 ' has': 7,
 ' a': 8,
 ' silver': 9,
 ' lining': 10,
 '<endoftext>': 11}

In [5]:
# create sequence of tokens
Sn_text = [[' I', ' think', ' there', 'fore', ' I', ' am', '<endoftext>'],
               ['Every', ' cloud', ' has', ' a', ' silver', ' lining', '<endoftext>']]
# turn into indices
Sn_indices = [[E_dict[word] for word in sentence] for sentence in Sn_text]
# turn into a tensor
Sn = torch.tensor(Sn_indices)
print("Sn: ", Sn.dtype, Sn.shape, Sn)

# starting off with the first token for each sequence
i=0
if i==0: 
    print(i)
    S_i = Sn[:,i].unsqueeze(dim=1) 
else: 
    print(i)
    S_i = Sn[:,0:i+1]
print("S_i: ", S_i.dtype, S_i.shape, S_i)

Sn:  torch.int64 torch.Size([2, 7]) tensor([[ 0,  1,  2,  3,  0,  4, 11],
        [ 5,  6,  7,  8,  9, 10, 11]])
0
S_i:  torch.int64 torch.Size([2, 1]) tensor([[0],
        [5]])


In [6]:
# embedding matrix
torch.manual_seed(seed)
E = torch.randn(v,d)

layer_norm = nn.LayerNorm(d)

# Apply layer normalization to the matrix
E = layer_norm(E)
print("E: ", E.shape, E)

# Look up the embeddings
X0 = F.embedding(S_i, E)
print("X0: ", X0.shape, X0)

E:  torch.Size([12, 4]) tensor([[-0.5100, -1.3183,  0.5075,  1.3208],
        [ 1.0348,  0.7062, -1.5466, -0.1943],
        [-1.3227, -0.5735,  0.6994,  1.1968],
        [-1.6470,  1.0348,  0.1860,  0.4262],
        [-1.2501,  1.2517, -0.6605,  0.6589],
        [-0.3892, -1.4613,  0.9034,  0.9471],
        [-1.2375,  0.7945,  1.1540, -0.7109],
        [-0.2447, -0.7559,  1.6978, -0.6973],
        [-0.3729, -0.4507,  1.6999, -0.8764],
        [ 1.6578, -0.3649, -1.0225, -0.2704],
        [ 0.0544, -1.6133,  1.0787,  0.4802],
        [-1.3672,  0.7058, -0.5100,  1.1714]],
       grad_fn=<NativeLayerNormBackward0>)
X0:  torch.Size([2, 1, 4]) tensor([[[-0.5100, -1.3183,  0.5075,  1.3208]],

        [[-0.3892, -1.4613,  0.9034,  0.9471]]], grad_fn=<EmbeddingBackward0>)


#### let's not actually code up the attention mechanism and say we did

In [7]:
# ffn layer & residual connection
torch.manual_seed(seed)
W1 = torch.randn(d,d**2)
X1 = torch.matmul(X0,W1)

relu = nn.ReLU()
X2 = relu(X1)

torch.manual_seed(seed)
W2 = torch.randn(d**2,d)
Xf = layer_norm(torch.matmul(X2,W2)+X0)

print("Xf: ", Xf.shape, Xf)

Xf:  torch.Size([2, 1, 4]) tensor([[[-0.7480, -1.0568,  0.3128,  1.4920]],

        [[-0.7732, -1.0541,  0.3562,  1.4711]]],
       grad_fn=<NativeLayerNormBackward0>)


# 2. regular GPT output layer

imma show how regular GPT does it just for reference when writing out mine later

In [8]:
Z = torch.matmul(Xf,E.T)
print("Z: ", Z.shape, Z)

Z:  torch.Size([2, 1, 12]) tensor([[[ 3.9039, -2.2939,  3.5998,  0.8324,  0.3887,  3.5309, -0.6137,
           0.4725, -0.0207, -1.5775,  2.7181,  1.8649]],

        [[ 3.9077, -2.3812,  3.6370,  0.8759,  0.3811,  3.5563, -0.5154,
           0.5650,  0.0796, -1.6590,  2.7493,  1.8547]]],
       grad_fn=<UnsafeViewBackward0>)


In [9]:
Z_prime = Z[:,-1,:]
print("Z_prime: ", Z_prime.shape, Z_prime)

Zn:  torch.Size([2, 12]) tensor([[ 3.9039, -2.2939,  3.5998,  0.8324,  0.3887,  3.5309, -0.6137,  0.4725,
         -0.0207, -1.5775,  2.7181,  1.8649],
        [ 3.9077, -2.3812,  3.6370,  0.8759,  0.3811,  3.5563, -0.5154,  0.5650,
          0.0796, -1.6590,  2.7493,  1.8547]], grad_fn=<SliceBackward0>)


# 3. regular GPT training

In [10]:
# targets
s_iplus1 = Sn[:,i+1]
print("s_iplus1: ", s_iplus1.dtype, s_iplus1.shape, s_iplus1)

s_iplus1:  torch.int64 torch.Size([2]) tensor([1, 6])


In [11]:
# cross-entropy loss
loss = F.cross_entropy(Z_prime, s_iplus1)
print("loss: ", loss.shape, loss)

loss:  torch.Size([]) tensor(6.4204, grad_fn=<NllLossBackward0>)


#### then do backprop

# 4. regular GPT inference

In [12]:
P = F.softmax(Zn, dim=-1)
print("P: ", P.shape, P)

P:  torch.Size([2, 12]) tensor([[0.3325, 0.0007, 0.2453, 0.0154, 0.0099, 0.2290, 0.0036, 0.0108, 0.0066,
         0.0014, 0.1016, 0.0433],
        [0.3267, 0.0006, 0.2492, 0.0158, 0.0096, 0.2299, 0.0039, 0.0115, 0.0071,
         0.0012, 0.1026, 0.0419]], grad_fn=<SoftmaxBackward0>)


In [13]:
s_iplus1 = torch.max(P, dim=1).indices
print("s_iplus1: ", s_iplus1.shape, s_iplus1)

s_iplus1:  torch.Size([2]) tensor([0, 0])


In [14]:
S_iplus1 = torch.cat((S_i, s_iplus1.unsqueeze(dim=1)), dim=1)
print("S_iplus1: ", S_iplus1.shape, S_iplus1)

S_iplus1:  torch.Size([2, 2]) tensor([[0, 0],
        [5, 0]])


# 5. my model's output layer

In [8]:
# select final row BEFORE multiplying by the embedding matrix
Y = Xf[:,-1:].squeeze(dim=1)
print("Y: ", Y.shape, Y)

Y:  torch.Size([2, 4]) tensor([[-0.7480, -1.0568,  0.3128,  1.4920],
        [-0.7732, -1.0541,  0.3562,  1.4711]], grad_fn=<SqueezeBackward1>)


in a normal GPT we'd first multiply by E.T and then select the row, because normal gpt's want to immediately select from a b,v tensor whereas we need a b,d tensor to give us concept vectors

In [9]:
Z = torch.matmul(Y,E.T)
print("Z: ", Z.shape, Z)

Z:  torch.Size([2, 12]) tensor([[ 3.9039, -2.2939,  3.5998,  0.8324,  0.3887,  3.5309, -0.6137,  0.4725,
         -0.0207, -1.5775,  2.7181,  1.8649],
        [ 3.9077, -2.3812,  3.6370,  0.8759,  0.3811,  3.5563, -0.5154,  0.5650,
          0.0796, -1.6590,  2.7493,  1.8547]], grad_fn=<MmBackward0>)


In [10]:
P = F.softmax(Z, dim=-1)
print("P: ", P.shape, P)

P:  torch.Size([2, 12]) tensor([[0.3325, 0.0007, 0.2453, 0.0154, 0.0099, 0.2290, 0.0036, 0.0108, 0.0066,
         0.0014, 0.1016, 0.0433],
        [0.3267, 0.0006, 0.2492, 0.0158, 0.0096, 0.2299, 0.0039, 0.0115, 0.0071,
         0.0012, 0.1026, 0.0419]], grad_fn=<SoftmaxBackward0>)


# 6. my model's conditional weird stuff

In [11]:
P_bar = torch.max(P, dim=1).values
print("P_bar: ", P_bar.shape, P_bar)

P_bar:  torch.Size([2]) tensor([0.3325, 0.3267], grad_fn=<MaxBackward0>)


In [12]:
# to guarantee we get interesting results
# in reality we'd set this as a hyperparameter
gamma0 = torch.mean(P_bar)
print("gamma0: ", gamma0)

Gamma_i, D_i = None, None

if i==0:
    Gamma_i = torch.ones(b)*gamma0
else:
    try:
        Gamma_i = Gamma_i + D_i
    except TypeError:
        print("Gamma_i and D_i are not defined. restart the sequence beginning at i=0")
        
    
try:
    print("Gamma_i: ", Gamma_i.shape, Gamma_i)
except:
    print("Gamma_i and D_i are not defined. restart the sequence beginning at i=0")

gamma0:  tensor(0.3296, grad_fn=<MeanBackward0>)
Gamma_i:  torch.Size([2]) tensor([0.3296, 0.3296], grad_fn=<MulBackward0>)


In [13]:
P_bar_mask = P_bar >= gamma0
print("P_bar_mask: ", P_bar_mask.shape, P_bar_mask)

A = torch.zeros(b)
A[P_bar_mask] = 1
print("A: ", A.dtype, A.shape, A)

P_bar_mask:  torch.Size([2]) tensor([ True, False])
A:  torch.float32 torch.Size([2]) tensor([1., 0.])


In [14]:
O = torch.ones(b)
print("O: ", O.dtype, O.shape, O)

A_prime = O-A
print("A_prime: ", A_prime.dtype, A_prime.shape, A_prime)

O:  torch.float32 torch.Size([2]) tensor([1., 1.])
A_prime:  torch.float32 torch.Size([2]) tensor([0., 1.])


In [15]:
# maximum allowable % increase in compute
Cmax = torch.tensor(.7) 

# resulting maximum allowable extra tokens (since attention is O(n^2))
tmax = torch.floor(torch.sqrt(Cmax*n))
print("tmax: ", tmax)

# resulting increment of gamma each time a concept vector is created
delta_gamma = (torch.tensor(1)-gamma0)/tmax
print("delta_gamma: ", delta_gamma)
# it shouldn't be differentiable, that's just an artifact of me setting gamma0 to the mean of P

tmax:  tensor(2.)
delta_gamma:  tensor(0.3352, grad_fn=<DivBackward0>)


In [16]:
D_i = A_prime * delta_gamma
print("D_i: ", D_i.shape, D_i)

D_i:  torch.Size([2]) tensor([0.0000, 0.3352], grad_fn=<MulBackward0>)


In [17]:
Gamma_iplus1 = Gamma_i + D_i
print("Gamma_iplus1: ", Gamma_iplus1.shape, Gamma_iplus1)

Gamma_iplus1:  torch.Size([2]) tensor([0.3296, 0.6648], grad_fn=<AddBackward0>)


# 7. training with my model

In [21]:
# targets
s_iplus1 = Sn[:,i+1]
print("s_iplus1: ", s_iplus1.dtype, s_iplus1.shape, s_iplus1)

s_iplus1:  torch.int64 torch.Size([2]) tensor([1, 6])


In [27]:
def custom_loss(logits, target, mask):
    
    assert isinstance(logits, torch.Tensor), "logits must be a PyTorch tensor"
    assert isinstance(target, torch.Tensor), "target must be a PyTorch tensor"
    assert isinstance(mask, torch.Tensor), "mask must be a PyTorch tensor"
    
    assert logits.shape == (b, v), f"logits must have shape {b} (batch), {v} (vocab), but has shape {logits.shape}"
    assert target.shape == torch.Size([b]), f"target must have shape {b} (batch), but has shape {target.shape}"
    assert mask.shape == torch.Size([b]), f"mask must have shape {b} (batch), but has shape {mask.shape}"
    
    # Apply LogSoftmax
    log_probs = F.log_softmax(logits, dim=1)
    print("log_probs: ", log_probs.shape, log_probs)
    
    # Gather the log probabilities corresponding to the true classes
    gathered_probs = log_probs.gather(1, target.unsqueeze(1)).squeeze(1)
    print("gathered_probs: ", gathered_probs.shape, gathered_probs)
    
    # Apply the mask
    masked_probs = gathered_probs * mask
    print("masked_probs: ", masked_probs.shape, masked_probs)

    # Compute the average loss over unmasked entries
    # Use A.sum() to count the number of unmasked entries
    loss = -torch.sum(masked_probs) / torch.clamp(mask.sum(), min=1)
    return loss

In [28]:
# Example usage
loss = custom_loss(Z, s_iplus1, A)
print("Custom Loss:", loss)

log_probs:  torch.Size([2, 12]) tensor([[-1.1011, -7.2988, -1.4052, -4.1725, -4.6162, -1.4741, -5.6186, -4.5325,
         -5.0257, -6.5824, -2.2869, -3.1400],
        [-1.1188, -7.4078, -1.3895, -4.1506, -4.6454, -1.4702, -5.5419, -4.4616,
         -4.9469, -6.6856, -2.2773, -3.1719]], grad_fn=<LogSoftmaxBackward0>)
gathered_probs:  torch.Size([2]) tensor([-7.2988, -5.5419], grad_fn=<SqueezeBackward1>)
masked_probs:  torch.Size([2]) tensor([-0.0000, -5.5419], grad_fn=<MulBackward0>)
Custom Loss: tensor(5.5419, grad_fn=<DivBackward0>)


In [26]:
# let's do regular cross-entropy for comparison (not gonna actually use it)
# manual-ish calc
log_softmax_output = F.log_softmax(Z, dim=1)
nll_loss_output = F.nll_loss(log_softmax_output, s_iplus1)
print("LogSoftmax Output:", log_softmax_output)
print("NLLLoss Output:", nll_loss_output)
# auto calc
cross_entropy_loss = F.cross_entropy(Z, s_iplus1)
print("CrossEntropy Loss:", cross_entropy_loss)

LogSoftmax Output: tensor([[-1.1011, -7.2988, -1.4052, -4.1725, -4.6162, -1.4741, -5.6186, -4.5325,
         -5.0257, -6.5824, -2.2869, -3.1400],
        [-1.1188, -7.4078, -1.3895, -4.1506, -4.6454, -1.4702, -5.5419, -4.4616,
         -4.9469, -6.6856, -2.2773, -3.1719]], grad_fn=<LogSoftmaxBackward0>)
NLLLoss Output: tensor(6.4204, grad_fn=<NllLossBackward0>)
CrossEntropy Loss: tensor(6.4204, grad_fn=<NllLossBackward0>)


# 8. inference using my model

#### 8a. first let's get the user their output tokens

In [19]:
# boring non-masked predicted indices
s_iplus1 = torch.max(P, dim=1).indices
print("s_iplus1: ", s_iplus1.shape, s_iplus1)

s_iplus1:  torch.Size([2]) tensor([0, 0])


In [39]:
# masked indices - the ones left will actually be shown to the user
s_iplus1_prime = s_iplus1
s_iplus1_prime[A_prime.bool()] = v
print("s_iplus1_prime: ", s_iplus1_prime.shape, s_iplus1_prime)
print(f"remember an index value of {v} is equal to an embedding vector of entirely zeros and the empty token ''")

s_iplus1_prime:  torch.Size([2]) tensor([ 0, 12])
remember an index value of 12 is equal to an embedding vector of entirely zeros and the empty token ''


In [40]:
# append to the current running sequence to give the user their output
S_iplus1 = torch.cat((S_i, s_iplus1_prime.unsqueeze(dim=1)),dim=1)
print("S_iplus1: ", S_iplus1.shape, S_iplus1)

S_iplus1:  torch.Size([2, 2]) tensor([[ 0,  0],
        [ 5, 12]])


#### that's the tokens that the user will see. notice how 12 is an empty token '' & embedding vector of 0's

#### 8b. now let's get what the model needs for the next round of inference

In [41]:
epsilon = torch.zeros(d)
print("epsilon aka the embedding vector of the empty token '': ", epsilon.shape, epsilon)

E_prime = torch.cat((E,epsilon.unsqueeze(0)), dim=0)
print("E_prime: ", E_prime.shape, E_prime)

epsilon aka the embedding vector of the empty token '':  torch.Size([4]) tensor([0., 0., 0., 0.])
E_prime:  torch.Size([13, 4]) tensor([[-0.5100, -1.3183,  0.5075,  1.3208],
        [ 1.0348,  0.7062, -1.5466, -0.1943],
        [-1.3227, -0.5735,  0.6994,  1.1968],
        [-1.6470,  1.0348,  0.1860,  0.4262],
        [-1.2501,  1.2517, -0.6605,  0.6589],
        [-0.3892, -1.4613,  0.9034,  0.9471],
        [-1.2375,  0.7945,  1.1540, -0.7109],
        [-0.2447, -0.7559,  1.6978, -0.6973],
        [-0.3729, -0.4507,  1.6999, -0.8764],
        [ 1.6578, -0.3649, -1.0225, -0.2704],
        [ 0.0544, -1.6133,  1.0787,  0.4802],
        [-1.3672,  0.7058, -0.5100,  1.1714],
        [ 0.0000,  0.0000,  0.0000,  0.0000]], grad_fn=<CatBackward0>)


In [42]:
Y_token = F.embedding(s_iplus1_prime, E_prime)
print("Y_token: ", Y_token.shape, Y_token)

Y_token:  torch.Size([2, 4]) tensor([[-0.5100, -1.3183,  0.5075,  1.3208],
        [ 0.0000,  0.0000,  0.0000,  0.0000]], grad_fn=<EmbeddingBackward0>)


In [44]:
A_prime_expand = A_prime.unsqueeze(dim=1)
print("A_prime_expand: ", A_prime_expand.shape, A_prime_expand)

Y_concept = Y*A_prime_expand
print("Y_concept: ", Y_concept.shape, Y_concept)

A_prime_expand:  torch.Size([2, 1]) tensor([[0.],
        [1.]])
Y_concept:  torch.Size([2, 4]) tensor([[-0.0000, -0.0000,  0.0000,  0.0000],
        [-0.7732, -1.0541,  0.3562,  1.4711]], grad_fn=<MulBackward0>)


In [45]:
Y_inference = Y_token + Y_concept
print("Y_inference: ", Y_inference.shape, Y_inference)

Y_inference:  torch.Size([2, 4]) tensor([[-0.5100, -1.3183,  0.5075,  1.3208],
        [-0.7732, -1.0541,  0.3562,  1.4711]], grad_fn=<AddBackward0>)


In [49]:
print("X0_i: ", X0.shape, X0)

Y_inference_prime = Y_inference.unsqueeze(dim=1)
print("Y_inference_prime: ", Y_inference_prime.shape, Y_inference_prime)

X0_iplus1 = torch.cat((X0, Y_inference_prime), dim=1)
print("X0_iplus1: ", X0_iplus1.shape, X0_iplus1)

X0_i:  torch.Size([2, 1, 4]) tensor([[[-0.5100, -1.3183,  0.5075,  1.3208]],

        [[-0.3892, -1.4613,  0.9034,  0.9471]]], grad_fn=<EmbeddingBackward0>)
Y_inference_prime:  torch.Size([2, 1, 4]) tensor([[[-0.5100, -1.3183,  0.5075,  1.3208]],

        [[-0.7732, -1.0541,  0.3562,  1.4711]]], grad_fn=<UnsqueezeBackward0>)
X0_iplus1:  torch.Size([2, 2, 4]) tensor([[[-0.5100, -1.3183,  0.5075,  1.3208],
         [-0.5100, -1.3183,  0.5075,  1.3208]],

        [[-0.3892, -1.4613,  0.9034,  0.9471],
         [-0.7732, -1.0541,  0.3562,  1.4711]]], grad_fn=<CatBackward0>)


#### and that's the residual state that the model gets to use on its next inference pass

# 9. turning it into useful functions

#### 9a. first we'll make the setup functions that both regular GPT's and our model use

In [3]:
def create_input_sequence(i, S_n_text):
    global verbose, seed
    
    # turn into indices
    S_n_indices = [[E_dict[word] for word in sentence] for sentence in S_n_text]
    # turn into a tensor
    S_n = torch.tensor(S_n_indices)
    if verbose:
        print("S_n: ", S_n.dtype, S_n.shape, S_n)

    # starting off with the first token for each sequence
    if i==0: 
        if verbose:
            print(i)
        S_i = S_n[:,i].unsqueeze(dim=1) 
    else: 
        if verbose:
            print(i)
        S_i = S_n[:,0:i+1]
        
    if verbose:
        print("S_i: ", S_i.dtype, Si.shape, S_i)
        
    return S_n, S_i

def create_embeddings(v, d):
    global verbose, seed, layer_norm
    
    # embedding matrix
    torch.manual_seed(seed)
    E = torch.randn(v,d)

    # Apply layer normalization to the matrix
    E = layer_norm(E)
    if verbose:
        print("E: ", E.shape, E)
        
    return E

def create_first_resid(S_i, E):
    global verbose
    
    # Look up the embeddings
    X0 = F.embedding(S_i, E)
    if verbose:
        print("X0: ", X0.shape, X0)
        
    return X0

#### 9b. the guts of the model, which we will not be editing

In [4]:
def mha(x, h):
    # fuck this i'm not editing attention so no need to mess with it
    return x

def ffn(X0):
    global verbose, seed
    """
    I know i didn't do mha but i think i might need a nonlinearity in here 
    so i guess i'll do the ffn
    """
    
    # first linear layer
    torch.manual_seed(seed)
    W1 = torch.randn(d,d**2)
    if verbose:
        print("W1: ", W1.shape, W1)
    X1 = torch.matmul(X0,W1)
    if verbose:
        print("X1: ", X1.shape, X1)

    # activation function
    relu = nn.ReLU()
    X2 = relu(X1)
    if verbose:
        print("X2: ", X2.shape, X2)

    # second linear layer
    torch.manual_seed(seed)
    W2 = torch.randn(d**2,d)
    if verbose:
        print("W2: ", W2.shape, W2)
    X3 = torch.matmul(X2,W2)
    if verbose:
        print("X3: ", X3.shape, X3)
    
    return X3
    
def resid_connection(X0, Xi):
    global verbose, layer_norm
    
    # can't remember when / how frequently you layernorm
    # too lazy to check, i don't think it matters
    Xf = layer_norm(Xi+X0)
    
    if verbose:
        print("Xf: ", Xf.shape, Xf)
        
    return Xf

#### 9c. the stuff a regular next-token predictor GPT uses which we will not be interested in. I still wrote it out for sake of demonstration tho

In [5]:
def gpt_output_layer(Xf, E):
    global verbose
    
    Z = torch.matmul(Xf,E.T)
    if verbose:
        print("E^T: ", E.T.shape, E.T)
        print("Z: ", Z.shape, Z)
    
    Z_prime = Z[:,-1,:]
    if verbose:
        print("Z_prime: ", Z_prime.shape, Z_prime)
    
    return Z_prime

def gpt_get_loss(S_n, i, Z_prime):
    global verbose
    
    # targets
    s_iplus1 = S_n[:,i+1]
    if verbose:
        print("s_iplus1: ", s_iplus1.dtype, s_iplus1.shape, s_iplus1)
    
    # cross-entropy loss
    loss = F.cross_entropy(Z_prime, s_iplus1)
    if verbose:
        print("loss: ", loss.shape, loss)
        
    return s_iplus1, loss

def gpt_perform_inference(Zn, S_i):
    global verbose
    
    P = F.softmax(Zn, dim=-1)
    if verbose:
        print("P: ", P.shape, P)

    s_iplus1 = torch.max(P, dim=1).indices
    if verbose:
        print("s_iplus1: ", s_iplus1.shape, s_iplus1)

    # capital S means whole sequence while lowercase s means just that timestep of tokens
    S_iplus1 = torch.cat((S_i, s_iplus1.unsqueeze(dim=1)), dim=1)
    if verbose:
        print("S_iplus1: ", S_iplus1.shape, S_iplus1)
        
    return P, s_iplus1, S_iplus1

In [6]:
# a function to give us a pretty read off of the sequences
def tensor_to_string(tensor, index_to_token):
    """
    Convert a PyTorch tensor of indices into readable strings using a mapping dictionary.

    :param tensor: PyTorch tensor of indices. Can be 1D or 2D.
    :param index_to_token: Dictionary mapping indices to tokens.
    :return: A single string if the tensor is 1D, or a list of strings if the tensor is 2D.
    """
    # Check if the tensor is 1D or 2D
    if len(tensor.shape) == 1:
        # Convert the 1D tensor to a single string
        return ' '.join([index_to_token[int(idx)] for idx in tensor])
    elif len(tensor.shape) == 2:
        # Convert the tensor to a list of lists and then to strings
        sequences = tensor.tolist()
        return [' '.join([index_to_token[int(idx)] for idx in seq]) for seq in sequences]
    else:
        raise ValueError("Tensor must be either 1D or 2D")

# Example usage
token_to_index = {'hello': 1, 'world': 2}
index_to_token = {v: k for k, v in token_to_index.items()}
S = torch.tensor([[1, 2], [2, 1]])  # Example tensor
S_single = torch.tensor([1, 2])     # Example 1D tensor

readable_strings_2d = tensor_to_string(S, index_to_token)
readable_string_1d = tensor_to_string(S_single, index_to_token)

print("2D tensor:", readable_strings_2d)
print("1D tensor:", readable_string_1d)

2D tensor: ['hello world', 'world hello']
1D tensor: hello world


#### 9d. my model's output layer

In [7]:
def gamma_model_output_layer(Xf, E):
    global verbose
    
    # select final row BEFORE multiplying by the embedding matrix
    Y = Xf[:,-1:].squeeze(dim=1)
    if verbose:
        print("Y: ", Y.shape, Y)

    Z = torch.matmul(Y,E.T)
    if verbose:
        print("Z: ", Z.shape, Z)

    P = F.softmax(Z, dim=-1)
    if verbose:
        print("P: ", P.shape, P)
        
    return Y, Z, P

#### 9e. my model's weird conditional logic

In [17]:
class SequenceError(Exception):
    pass

def initialize_Gamma_and_D(i):
    if i!=0:
        raise SequenceError("This function should only be called when i=0, aka at the beginning of the batch/sequence")
    else:
        Gamma_i = torch.ones(b)*gamma0
    
    if verbose:
        print("Gamma_i: ", Gamma_i.shape, Gamma_i)
        
    return Gamma_i

def iterate_Gamma_and_D(i, Gamma_i, D_i):
    if i<=0:
        raise SequenceError("This function should only be called when i>0, aka NOT at the beginning of the batch/sequence")
    else:
        Gamma_i = torch.clamp(Gamma_i + D_i, max=1)
    
    if verbose:
        print("Gamma_i: ", Gamma_i.shape, Gamma_i)
        
    return Gamma_i
   
def creating_conditionals(P, gamma0):
    
    P_bar = torch.max(P, dim=1).values
    if verbose:
        print("P_bar: ", P_bar.shape, P_bar)

    P_bar_mask = P_bar >= gamma0
    if verbose:
        print("P_bar_mask: ", P_bar_mask.shape, P_bar_mask)

    A = torch.zeros(b)
    A[P_bar_mask] = 1
    if verbose:
        print("A: ", A.dtype, A.shape, A)

    O = torch.ones(b)
    if verbose:
        print("O: ", O.dtype, O.shape, O)

    A_prime = O-A
    if verbose:
        print("A_prime: ", A_prime.dtype, A_prime.shape, A_prime)
        
    return A, O, A_prime

def calc_delta_gamma(Cmax, n, A_prime):
    
    # resulting maximum allowable extra tokens (since attention is O(n^2))
    tmax = torch.floor(torch.sqrt(Cmax*n))
    if verbose:
        print("tmax: ", tmax)

    # resulting increment of gamma each time a concept vector is created
    delta_gamma = (torch.tensor(1)-gamma0)/tmax
    if verbose:
        print("delta_gamma: ", delta_gamma)
    
    D_i = A_prime * delta_gamma
    if verbose:
        print("D_i: ", D_i.shape, D_i)

    return delta_gamma, D_i

#### 9f. training w/ my model

In [13]:
def get_target_tokens(i, S_n):
    global verbose
    
    # targets
    s_iplus1 = S_n[:,i+1]
    
    if verbose:
        print("s_iplus1: ", s_iplus1.dtype, s_iplus1.shape, s_iplus1)
        
    return s_iplus1

def custom_loss(logits, target, mask):
    
    assert isinstance(logits, torch.Tensor), "logits must be a PyTorch tensor"
    assert isinstance(target, torch.Tensor), "target must be a PyTorch tensor"
    assert isinstance(mask, torch.Tensor), "mask must be a PyTorch tensor"
    
    assert logits.shape == (b, v), f"logits must have shape {b} (batch), {v} (vocab), but has shape {logits.shape}"
    assert target.shape == torch.Size([b]), f"target must have shape {b} (batch), but has shape {target.shape}"
    assert mask.shape == torch.Size([b]), f"mask must have shape {b} (batch), but has shape {mask.shape}"
    
    # Apply LogSoftmax
    log_probs = F.log_softmax(logits, dim=1)
    if verbose:
        print("log_probs: ", log_probs.shape, log_probs)
    
    # Gather the log probabilities corresponding to the true classes
    gathered_probs = log_probs.gather(1, target.unsqueeze(1)).squeeze(1)
    if verbose:
        print("gathered_probs: ", gathered_probs.shape, gathered_probs)
    
    # Apply the mask
    masked_probs = gathered_probs * mask
    if verbose:
        print("masked_probs: ", masked_probs.shape, masked_probs)

    # Compute the average loss over unmasked entries
    # Use A.sum() to count the number of unmasked entries
    loss = -torch.sum(masked_probs) / torch.clamp(mask.sum(), min=1)
    if verbose:
        print("Custom Loss:", loss)
        
    return loss

# Example usage
#loss = custom_loss(Z, s_iplus1, A_prime)

#### 9g. inference w/ my model

In [10]:
def make_tokens_that_user_sees(P, A_prime, v, S_i):
    global verbose

    # boring non-masked predicted indices from inference
    s_iplus1 = torch.max(P, dim=1).indices
    if verbose:
        print("s_iplus1: ", s_iplus1.shape, s_iplus1)

    # masked indices - the ones left will actually be shown to the user
    s_iplus1_prime = s_iplus1
    s_iplus1_prime[A_prime.bool()] = v
    if verbose:
        print("s_iplus1_prime: ", s_iplus1_prime.shape, s_iplus1_prime)
        print(f"remember an index value of {v} is equal to an embedding vector of entirely zeros and the empty token ''")

    # append to the current running sequence to give the user their output
    S_iplus1 = torch.cat((S_i, s_iplus1_prime.unsqueeze(dim=1)),dim=1)
    if verbose:
        print("S_iplus1: ", S_iplus1.shape, S_iplus1)
        
    return s_iplus1_prime, S_iplus1

def make_next_input_for_model(d, E, s_iplus1_prime, A_prime, Y, X0):
    global verbose
    
    epsilon = torch.zeros(d)
    if verbose:
        print("epsilon aka the embedding vector of the empty token '': ", epsilon.shape, epsilon)

    E_prime = torch.cat((E,epsilon.unsqueeze(0)), dim=0)
    if verbose:
        print("E_prime: ", E_prime.shape, E_prime)

    Y_token = F.embedding(s_iplus1_prime, E_prime)
    if verbose:
        print("Y_token: ", Y_token.shape, Y_token)

    A_prime_expand = A_prime.unsqueeze(dim=1)
    if verbose:
        print("A_prime_expand: ", A_prime_expand.shape, A_prime_expand)

    Y_concept = Y*A_prime_expand
    if verbose:
        print("Y_concept: ", Y_concept.shape, Y_concept)

    Y_inference = Y_token + Y_concept
    if verbose:
        print("Y_inference: ", Y_inference.shape, Y_inference)
        print("X0_i: ", X0.shape, X0)

    Y_inference_prime = Y_inference.unsqueeze(dim=1)
    if verbose:
        print("Y_inference_prime: ", Y_inference_prime.shape, Y_inference_prime)

    X0_iplus1 = torch.cat((X0, Y_inference_prime), dim=1)
    if verbose:
        print("X0_iplus1: ", X0_iplus1.shape, X0_iplus1)
        
    return X0_iplus1

# 10. making a GPT training loop

In [18]:
# setting regular hyperparameters
b=2
n=7
d=4
v=12
layers = 2
h=4

layer_norm = nn.LayerNorm(d)

# our tokens being mapped to indices. obvi a stupidly minimally small vocab
E_dict = {" I":0,
          " think":1,
         " there":2,
         "fore":3,
         " am":4,
         "Every":5,
         " cloud":6,
         " has":7,
         " a":8,
         " silver":9,
         " lining":10,
         "<endoftext>":11}

S_n_text = [[' I', ' think', ' there', 'fore', ' I', ' am', '<endoftext>'],
               ['Every', ' cloud', ' has', ' a', ' silver', ' lining', '<endoftext>']]

# ofc
seed = 42069

# to print literally everything
verbose = False

In [29]:
# training loop
for i in range(n-1):
    
    print(f"i = {i}")
    
    # initialize everything
    if i==0:
        E = create_embeddings(v, d)
    
    S_n, S_i = create_input_sequence(i, S_n_text)
    
    E_dict_invert = {g: h for h, g in E_dict.items()}
    readable_strings = tensor_to_string(S_i, E_dict_invert)
    print("context S_i: ", readable_strings)
    
    X0 = create_first_resid(S_i, E)
    
    for l in range(layers):
        
        # multi-head attention, feedforward network, & residual connection
        if l== 0: # first layer
            Xi = mha(X0, h)
            Xi = ffn(Xi)
            Xi = resid_connection(X0, Xi)
            
        elif l == layers-1: # last layer
            X_iplus1 = mha(Xi, h)
            X_iplus1 = ffn(X_iplus1)
            Xf = resid_connection(Xi, X_iplus1)
            
        else: # all layers in the middle
            X_iplus1 = mha(Xi, h)
            X_iplus1 = ffn(X_iplus1)
            Xi = resid_connection(Xi, X_iplus1)
            
    Z_prime = gpt_output_layer(Xf, E)

    s_iplus1, loss = gpt_get_loss(S_n, i, Z_prime)
    
    readable_strings = tensor_to_string(s_iplus1, E_dict_invert)
    print("target s_iplus1: ", readable_strings)
        
    print("loss: ", loss.shape, loss)

i = 0
context S_i:  [' I', 'Every']
target s_iplus1:   think  cloud
loss:  torch.Size([]) tensor(2.0714, grad_fn=<NllLossBackward0>)
i = 1
context S_i:  [' I  think', 'Every  cloud']
target s_iplus1:   there  has
loss:  torch.Size([]) tensor(5.4166, grad_fn=<NllLossBackward0>)
i = 2
context S_i:  [' I  think  there', 'Every  cloud  has']
target s_iplus1:  fore  a
loss:  torch.Size([]) tensor(5.7736, grad_fn=<NllLossBackward0>)
i = 3
context S_i:  [' I  think  there fore', 'Every  cloud  has  a']
target s_iplus1:   I  silver
loss:  torch.Size([]) tensor(2.4356, grad_fn=<NllLossBackward0>)
i = 4
context S_i:  [' I  think  there fore  I', 'Every  cloud  has  a  silver']
target s_iplus1:   am  lining
loss:  torch.Size([]) tensor(5.7470, grad_fn=<NllLossBackward0>)
i = 5
context S_i:  [' I  think  there fore  I  am', 'Every  cloud  has  a  silver  lining']
target s_iplus1:  <endoftext> <endoftext>
loss:  torch.Size([]) tensor(4.1991, grad_fn=<NllLossBackward0>)


# 11. making a GPT inference loop

In [9]:
# setting regular hyperparameters
b=1
n=7
d=4
v=12
layers = 2
h=4

layer_norm = nn.LayerNorm(d)

# our tokens being mapped to indices. obvi a stupidly minimally small vocab
E_dict = {" I":0,
          " think":1,
         " there":2,
         "fore":3,
         " am":4,
         "Every":5,
         " cloud":6,
         " has":7,
         " a":8,
         " silver":9,
         " lining":10,
         "<endoftext>":11}

S_n_text = [[' I', ' think', ' there', 'fore', ' I', ' am', '<endoftext>']]

# ofc
seed = 42069

# to print literally everything
verbose = False

In [11]:
# inference loop

for i in range(n-1):
    
    print(f"i = {i}")
    
    # initialize everything
    if i==0:
        E = create_embeddings(v, d)
        S_n, S_i = create_input_sequence(i, S_n_text)
    
    E_dict_invert = {g: h for h, g in E_dict.items()}
    readable_strings = tensor_to_string(S_i, E_dict_invert)
    print("context S_i: ", readable_strings)
    
    X0 = create_first_resid(S_i, E)
    
    for l in range(layers):
        
        # multi-head attention, feedforward network, & residual connection
        if l== 0: # first layer
            Xi = mha(X0, h)
            Xi = ffn(Xi)
            Xi = resid_connection(X0, Xi)
            
        elif l == layers-1: # last layer
            X_iplus1 = mha(Xi, h)
            X_iplus1 = ffn(X_iplus1)
            Xf = resid_connection(Xi, X_iplus1)
            
        else: # all layers in the middle
            X_iplus1 = mha(Xi, h)
            X_iplus1 = ffn(X_iplus1)
            Xi = resid_connection(Xi, X_iplus1)
            
    Z_prime = gpt_output_layer(Xf, E)

    P, s_iplus1, S_iplus1 = gpt_perform_inference(Z_prime, S_i)
    
    readable_strings = tensor_to_string(s_iplus1, E_dict_invert)
    print("predicted s_iplus1: ", readable_strings)
    
    readable_strings = tensor_to_string(S_iplus1, E_dict_invert)
    print("new total text visible to user: ", readable_strings)
    
    S_i = S_iplus1

i = 0
context S_i:  [' I']
predicted s_iplus1:   silver
new total text visible to user:  [' I  silver']
i = 1
context S_i:  [' I  silver']
predicted s_iplus1:   silver
new total text visible to user:  [' I  silver  silver']
i = 2
context S_i:  [' I  silver  silver']
predicted s_iplus1:   silver
new total text visible to user:  [' I  silver  silver  silver']
i = 3
context S_i:  [' I  silver  silver  silver']
predicted s_iplus1:   silver
new total text visible to user:  [' I  silver  silver  silver  silver']
i = 4
context S_i:  [' I  silver  silver  silver  silver']
predicted s_iplus1:   silver
new total text visible to user:  [' I  silver  silver  silver  silver  silver']
i = 5
context S_i:  [' I  silver  silver  silver  silver  silver']
predicted s_iplus1:   silver
new total text visible to user:  [' I  silver  silver  silver  silver  silver  silver']


#### obvi this has not gone through any training so repetition isn't surprising. also i didn't implement a way for it to finish at \<endoftext> bc i didn't realistically expect it to end then

# 12. gamma-model training loop

In [49]:
# setting regular hyperparameters
b=2
n=7
d=4
v=12
layers = 4
h=4

layer_norm = nn.LayerNorm(d)

# our tokens being mapped to indices. obvi a stupidly minimally small vocab
E_dict = {" I":0,
          " think":1,
         " there":2,
         "fore":3,
         " am":4,
         "Every":5,
         " cloud":6,
         " has":7,
         " a":8,
         " silver":9,
         " lining":10,
         "<endoftext>":11}

S_n_text = [[' I', ' think', ' there', 'fore', ' I', ' am', '<endoftext>'],
               ['Every', ' cloud', ' has', ' a', ' silver', ' lining', '<endoftext>']]

# ofc
seed = 42069

# to print literally everything
verbose = False

# hyperparameter we'll have to tune
gamma0 = 0.244

# maximum allowable % increase in compute
Cmax = torch.tensor(.7)

#### currently this setup doesn't actually create the next X0; likely among many other issues

In [50]:
# training loop

E_dict_invert = {g: h for h, g in E_dict.items()}

E_dict_prime = E_dict
E_dict_prime[''] = 12
E_dict_prime_invert = {g: h for h, g in E_dict_prime.items()}

for i in range(n-1):
    
    print(f"i = {i}")
    
    # initialize everything
    if i==0:
        E = create_embeddings(v, d)
        Gamma_i = initialize_Gamma_and_D(i)
        S_n, S_i = create_input_sequence(i, S_n_text)
        X0 = create_first_resid(S_i, E)
    else:
        Gamma_i = iterate_Gamma_and_D(i, Gamma_i, D_i)
        
        # initialize S_iminus1 (not actually correct but simpler)
        S_n, S_iminus1 = create_input_sequence(i-1, S_n_text)
        # create s_i
        s_i = s_iplus1_prime
        
        # create Y_token with s_i and E_prime
        # create Y_concept by masking Xf with A_prime_expand
        S_i = 
        X0 = X0_iplus1
        
    print("Gamma_i: ", Gamma_i.shape, Gamma_i)
    
    # need to have a way to iterate S_i and X0, but i think that goes down below
    
    readable_strings = tensor_to_string(S_i, E_dict_invert)
    print("context S_i: ", readable_strings)
    
    
    
    for l in range(layers):
        
        # multi-head attention, feedforward network, & residual connection
        if l== 0: # first layer
            Xi = mha(X0, h)
            Xi = ffn(Xi)
            Xi = resid_connection(X0, Xi)
            
        elif l == layers-1: # last layer
            X_iplus1 = mha(Xi, h)
            X_iplus1 = ffn(X_iplus1)
            Xf = resid_connection(Xi, X_iplus1)
            
        else: # all layers in the middle
            X_iplus1 = mha(Xi, h)
            X_iplus1 = ffn(X_iplus1)
            Xi = resid_connection(Xi, X_iplus1)
         
    Y, Z, P = gamma_model_output_layer(Xf, E)
    
    A, O, A_prime = creating_conditionals(P, gamma0)
    print("A: ", A.shape, A)
    
    s_iplus1 = get_target_tokens(i, S_n)
    
    readable_strings = tensor_to_string(s_iplus1, E_dict_prime_invert)
    print("theoretical target s_iplus1: ", readable_strings)
    
    loss = custom_loss(Z, s_iplus1, A_prime)
    print("loss: ", loss.shape, loss)
    
    _, D_i = calc_delta_gamma(Cmax, n, A_prime)
    print("D_i: ", D_i.shape, D_i)
    
    s_iplus1_prime = s_iplus1 * 
    print("s_iplus1_prime: ", s_iplus1_prime.shape, s_iplus1_prime)

    X0_iplus1 = make_next_input_for_model(d, E, s_iplus1_prime, A_prime, Y, X0)
    print("X0_iplus1: ", X0_iplus1.shape, X0_iplus1)

i = 0
Gamma_i:  torch.Size([2]) tensor([0.2440, 0.2440])
context S_i:  [' I', 'Every']
A:  torch.Size([2]) tensor([0., 1.])
target s_iplus1:   think  cloud
loss:  torch.Size([]) tensor(2.6002, grad_fn=<DivBackward0>)
D_i:  torch.Size([2]) tensor([0.3780, 0.0000])
i = 1
Gamma_i:  torch.Size([2]) tensor([0.2440, 0.2440])
context S_i:  [' I', 'Every']
A:  torch.Size([2]) tensor([0., 1.])
target s_iplus1:   there  has
loss:  torch.Size([]) tensor(7.1135, grad_fn=<DivBackward0>)
D_i:  torch.Size([2]) tensor([0.3780, 0.0000])
i = 2
Gamma_i:  torch.Size([2]) tensor([0.6220, 0.2440])
context S_i:  [' I', 'Every']
A:  torch.Size([2]) tensor([0., 1.])
target s_iplus1:  fore  a
loss:  torch.Size([]) tensor(2.0648, grad_fn=<DivBackward0>)
D_i:  torch.Size([2]) tensor([0.3780, 0.0000])
i = 3
Gamma_i:  torch.Size([2]) tensor([1.0000, 0.2440])
context S_i:  [' I', 'Every']
A:  torch.Size([2]) tensor([0., 1.])
target s_iplus1:   I  silver
loss:  torch.Size([]) tensor(3.2351, grad_fn=<DivBackward0>)
D_